In [1]:
"""
Excel to csv.

Makes a csv from excel files.
"""

import pandas as pd
import sys
import os
from tqdm import tqdm
import threading
from dotenv import dotenv_values
import requests
import boto3
import time


In [2]:
env_vars = dotenv_values('../.env')

In [3]:
def to_pandas(inf, outf):
    """
    To pandas.

    Converts from excel to pandas.
    """
    # read excel file
    df = pd.read_excel(inf)
    extra = df.columns[4:]
    df['extra'] = df[extra].apply(lambda row:
                                  ','.join(row.dropna().astype(str)), axis=1)
    df.drop(columns=extra, inplace=True)
    df.to_csv(outf, index=False)

In [4]:
def clean_csv(inf, outf):
    """
    Clean csv.

    Removes all unwanted data.
    """
    df = pd.read_csv(inf)
    df_filtered = df[df['label'].str.lower() != 'unmapped']
    df_filtered.to_csv(outf, index=False)

In [15]:
api_key = env_vars['CUSTOM_SEARCH_API_KEY']
search_engine_id = env_vars['SEARCH_ENGINE_ID']

def get_search(term):
    url = f'https://www.googleapis.com/customsearch/v1?key={api_key}&cx={search_engine_id}&q={term}'
    response = requests.get(url)
    data = response.json()

    res = ""
    if 'searchInformation' in data:
        if data['searchInformation']['totalResults'] == "0":
            return ""
    if 'items' in data:
        for item in data['items']:
            if 'title' in item:
                res += item['title'] + " "
            if 'snippet' in item: 
                res += item['snippet']+ " "
    if res == "":
        time.sleep(1)
        return get_search(term)
    return res

get_search("google")

"Google Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking\xa0... Google Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking\xa0... Google Cloud: Cloud Computing Services Meet your business challenges head on with cloud computing services from Google, including data management, hybrid & multi-cloud, and AI & ML. Google Account In your Google Account, you can see and manage your info, activity, security options, and privacy preferences to make Google work better for you. Google Analytics Google Analytics lets you measure your advertising ROI as well as track your Flash, video, and social networking sites and applications. Google Maps Find local businesses, view maps and get driving directions in Google Maps. Google Ads - Get Customers and Sell More with Online Advertising Let Google'

In [6]:
session = boto3.Session(
    aws_access_key_id=env_vars['AWS_API_KEY'],
    aws_secret_access_key=env_vars['AWS_API_SECRET'],
    region_name=env_vars['AWS_REGION']
)

dynamodb_client = session.client('dynamodb')

In [7]:
def get_table(term):
    try:
        key = {
            'name': {'S': term}
        }
        response = dynamodb_client.get_item(
            TableName=env_vars['AWS_TABLE_NAME'],
            Key=key
        )
        item = response.get('Item')
        if item:
            return item["content"]["S"]
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
print(get_table("google"))

Google Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking ... Google Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking ... Google Cloud: Cloud Computing Services Meet your business challenges head on with cloud computing services from Google, including data management, hybrid & multi-cloud, and AI & ML. Google Account In your Google Account, you can see and manage your info, activity, security options, and privacy preferences to make Google work better for you. Google Analytics Google Analytics lets you measure your advertising ROI as well as track your Flash, video, and social networking sites and applications. Google Maps Find local businesses, view maps and get driving directions in Google Maps. Google Ads - Get Customers and Sell More with Online Advertising Let Google's AI fi

In [8]:
def put_table(term, content):
    try:
        item = {
            'name': {'S': term},
            'content': {'S': content},
        }
        response = dynamodb_client.put_item(
            TableName=env_vars['AWS_TABLE_NAME'],
            Item=item
        )
    except Exception as e:
        print(f"An error occurred: {e}")
put_table("google", get_search("google"))

In [9]:
def get_object(vendor):
    if not vendor:
        return None
    vendor = str(vendor)
    table_res = get_table(vendor)
    if table_res:
        return table_res
    search_res = get_search(vendor)
    put_table(vendor, search_res)
    return search_res

In [10]:
def add_search(inf, outf):
    df = pd.read_csv(inf)
    
    overall_bar = tqdm(total=len(df), desc=f"Processing {inf}", position=0)
    
    def apply_with_progress(row):
        result = get_object(row['vendor'])
        overall_bar.update(1)
        return result
    
    df['search'] = df.apply(apply_with_progress, axis=1)
    
    overall_bar.close()
    df.to_csv(outf, index=False)


In [11]:
def iterate(in_dir, out_dir, function):
    """
    Iterate with tqdm.

    Iterates through files in a directory and uses tqdm for progress tracking.
    """
    threads = []
    for file in tqdm(os.listdir(in_dir), desc="Files", position=0):
        in_file = os.path.join(in_dir, file)
        out_file = os.path.join(out_dir, file)
        thread = threading.Thread(target=function, args=(in_file, out_file))
        threads.append(thread)
    
    with tqdm(total=len(threads), desc="Threads", position=1) as pbar:
        for thread in threads:
            thread.start()
        
        for thread in threads:
            thread.join()
            pbar.update(1)

In [12]:
def iterate_serial(in_dir, out_dir, function):
    for file in tqdm(os.listdir(in_dir), desc="Files", position=0):
        in_file = os.path.join(in_dir, file)
        out_file = os.path.join(out_dir, file)
        if not os.path.exists:
            function(in_file, out_file)

In [ ]:
iterate("../data/excel", "../data/csv", to_pandas)

In [ ]:
iterate("../data/csv", "../data/clean", clean_csv)

In [16]:
iterate_serial("../data/clean", "../data/search", add_search)

Files:   3%|███                                                                                                           | 3/107 [30:38<17:42:03, 612.72s/it]


KeyboardInterrupt: 

In [ ]:
for i in range(num_threads):
    thread = threading.Thread(target=worker_function, args=(i,))
    threads.append(thread)

# Start the threads
for thread in threads:
    thread.start()

# Use tqdm to track thread progress and wait for threads to finish
with tqdm(total=num_threads) as pbar:
    for thread in threads:
        thread.join()
        pbar.update(1)

In [17]:
s3_client = session.client('s3')


def sync_s3(dir):
    """
    Sync s3.

    Syncs data folder to s3.
    """
    try:
        s3_client.sync_dir(dir, env_vars['AWS_DATA_BUCKET'])
    except Exception as e:
        print(f"An error occurred: {e}")
sync_s3("../data/")

Processing ../data/clean/102.csv:  56%|█████████████████████████████████████████████▎                                   | 365/652 [3:08:36<2:28:18, 31.00s/it]

An error occurred: 'S3' object has no attribute 'sync_dir'


In [ ]:
bo